In [1]:
from basic import *
from helper import *
import numpy as np

In [2]:
def load_verysmallnn_weights(path: str = "verysmallnn_weights.npz"):
    """
    Load VerySmallNN weights saved via:
    np.savez('verysmallnn_weights.npz',
             W1=weights[0], b1=biases[0],
             W2=weights[1], b2=biases[1],
             W3=weights[2], b3=biases[2])
    """
    data = np.load(path)
    W1, b1 = data["W1"], data["b1"]
    W2, b2 = data["W2"], data["b2"]
    W3, b3 = data["W3"], data["b3"]
    weights = [W1, W2, W3]
    biases  = [b1, b2, b3]
    return weights, biases

In [3]:
def build_verysmallnn_polytope(
    lb_in: np.ndarray,
    ub_in: np.ndarray,
    pre_bounds: List[Tuple[np.ndarray, np.ndarray]],
    envelope,
    weights_path: str = "verysmallnn_weights.npz"
) -> PyomoPolyAnalyzer:
    """
    Build the PyomoPolyAnalyzer for VerySmallNN using saved weights.
    
    Assumes architecture:
      x_in -> (W1,b1) -> a1 -> ReLU -> z1
           -> (W2,b2) -> a2 -> ReLU -> z2
           -> (W3,b3) -> a3 (output, linear)
    
    pre_bounds: [ (L1, U1), (L2, U2) ] for a1 and a2 respectively.
    """
    weights, biases = load_verysmallnn_weights(weights_path)
    W1, W2, W3 = weights
    b1, b2, b3 = biases

    analyzer = PyomoPolyAnalyzer()

    # Input layer
    x0 = analyzer.add_input_box("x0", lb_in, ub_in)

    # Layer 1: x0 -> a1 -> z1
    (L1, U1), (L2, U2) = pre_bounds

    a1 = analyzer.add_affine(x0, W1, b1, "a1")
    z1 = analyzer.add_activation(a1, "z1", (L1, U1), envelope)

    # Layer 2: z1 -> a2 -> z2
    a2 = analyzer.add_affine(z1, W2, b2, "a2")
    z2 = analyzer.add_activation(a2, "z2", (L2, U2), envelope)

    # Output layer: z2 -> a3
    a3 = analyzer.add_affine(z2, W3, b3, "a3")

    return analyzer

In [4]:
weights, biases = load_verysmallnn_weights("verysmallnn_weights.npz")
W1, W2, W3 = weights
b1, b2, b3 = biases

in_dim = W1.shape[0]   # 49
h1     = W1.shape[1]   # 3
h2     = W2.shape[1]   # 3
out    = W3.shape[1]   # 10

# Input box in 49D (you can tweak these)
lb_in = np.zeros(in_dim)
ub_in = np.ones(in_dim)

# Crude pre-activation bounds, matching hidden sizes
L1 = -10 * np.ones(h1)   # length 3
U1 =  10 * np.ones(h1)
L2 = -10 * np.ones(h2)   # length 3
U2 =  10 * np.ones(h2)


In [5]:
analyzer = build_verysmallnn_polytope(
    lb_in, ub_in,
    pre_bounds=[(L1, U1), (L2, U2)],
    weights_path="verysmallnn_weights.npz",
    envelope=relu_envelope
)

model = analyzer.model

# Example: maximize first output logit a3[0]
obj_expr = model.a3[0]
val, res = analyzer.optimize(obj_expr, sense="max")
print("Max value of output[0]:", val)


Max value of output[0]: 22.5722446709595


In [7]:
weights, biases = load_verysmallnn_weights("verysmallnn_weights.npz")
W1, W2, W3 = weights
b1, b2, b3 = biases

# Get dimensions
in_dim = W1.shape[0]  # 49

# --- 1. Run Interval Bound Propagation (IBP) ---
print("Running IBP to find tight pre-activation bounds...")

# Define the input box [0, 1] for all 49 pixels
lb_in = np.zeros(in_dim)
ub_in = np.ones(in_dim)

# Propagate bounds using the Keras-compatible IBP function
L1, U1 = ibp_affine_keras(lb_in, ub_in, W1, b1)
Lz1, Uz1 = ibp_activation(L1, U1, gelu)

L2, U2 = ibp_affine_keras(Lz1, Uz1, W2, b2)
Lz2, Uz2 = ibp_activation(L2, U2, gelu)

# (No bounds needed for final logits 'a3')
print("IBP complete.")
print(f"Bounds for a1: L={np.min(L1):.2f}, U={np.max(U1):.2f}")
print(f"Bounds for a2: L={np.min(L2):.2f}, U={np.max(U2):.2f}")

# This list will be passed to the builder
pre_bounds = [(L1, U1), (L2, U2)]

Running IBP to find tight pre-activation bounds...
IBP complete.
Bounds for a1: L=-23.02, U=34.88
Bounds for a2: L=-47.71, U=63.35


In [8]:
analyzer = build_verysmallnn_polytope(
    lb_in, ub_in,
    pre_bounds=pre_bounds,
    weights_path="verysmallnn_weights.npz",
    envelope=tight_gelu_envelope 
)

model = analyzer.model

print("Optimizing for max value of logit 0...")
obj_expr = model.a3[0]
val, res = analyzer.optimize(obj_expr, sense="max")

if val is not None:
    print(f"Max value of output[0]: {val:.4f}")
else:
    print("Optimization failed.")

Optimizing for max value of logit 0...
Max value of output[0]: 301.3727
